In [1]:
from eeg_data_load import load_eeg_data
class_labels = [
    "Normal",
    "Complex_Partial_Seizures",
    "Electrographic_Seizures",
    "Video_detected_Seizures_with_no_visual_change_over_EEG",
]

train_path = "/kaggle/input/eeg-data-final/EEG_Data/train_data"
val_path = "/kaggle/input/eeg-data-final/EEG_Data/validation_data"

In [45]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.fft import fft

In [2]:
X_train, y_train = load_eeg_data(train_path, class_labels)
X_val, y_val = load_eeg_data(val_path, class_labels)

In [7]:
# Function to compute Fourier Transform
def compute_fourier_transform(data):
    fourier_transform = np.abs(fft(data, axis=1))
    return fourier_transform

# Function to compute Zero Crossing Rate
def compute_zero_crossing_rate(data):
    zero_crossing_rate = np.mean(np.diff(np.sign(data)) != 0, axis=1)
    return zero_crossing_rate

In [26]:
# Extract features for training set
fourier_features_train = compute_fourier_transform(X_train).mean(axis=(1, 2))  
zero_crossing_rates_train = compute_zero_crossing_rate(X_train).mean(axis=1)  

# Extract features for validation set
fourier_features_val = compute_fourier_transform(X_val).mean(axis=(1, 2)) 
zero_crossing_rates_val = compute_zero_crossing_rate(X_val).mean(axis=1)  

In [27]:
# Combine features for training set
features_train = np.hstack((fourier_features_train.reshape(-1, 1), zero_crossing_rates_train.reshape(-1, 1)))

# Combine features for validation set
features_val = np.hstack((fourier_features_val.reshape(-1, 1), zero_crossing_rates_val.reshape(-1, 1)))

In [29]:
# Standardize the features
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_val = scaler.transform(features_val)

In [32]:
# Train the SVM model
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(features_train, y_train)

SVC(kernel='linear', probability=True)

In [34]:
# Evaluate the model
y_pred = svm_model.predict(features_val)
y_prob = svm_model.predict_proba(features_val)

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
# Classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.92      0.75       696
           1       0.76      0.52      0.61       549
           2       0.00      0.00      0.00       137
           3       0.61      0.52      0.56        21

    accuracy                           0.67      1403
   macro avg       0.50      0.49      0.48      1403
weighted avg       0.62      0.67      0.62      1403



In [44]:
roc_auc = roc_auc_score(y_val, y_prob, multi_class='ovr')
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.770343056524569


In [46]:
# Balanced accuracy
balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
print(f"Balanced Accuracy: {balanced_accuracy}")

Balanced Accuracy: 0.4901634857824796
